In [ ]:
import requests

url = "https://api.sportradar.com/tennis/trial/v3/en/double_competitors_rankings.json?api_key=yymk81Bo0odf7J5MSnJ6rDrICbfCUjaQkfmVM7bc"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.text)


In [ ]:
json_data = response.text

json_data

In [ ]:
import pandas as pd
import json
import mysql.connector

# Convert JSON string to a Python dictionary
data = json.loads(json_data)

# Extract rankings (table 1)
df_rankings = pd.json_normalize(data["rankings"], 
                                record_path=["competitor_rankings"], 
                                meta=["year", "week"], 
                                record_prefix="rank_")

# Extract competitors (table 2)
df_competitors = df_rankings[["rank_competitor.id", "rank_competitor.name", 
                              "rank_competitor.country", "rank_competitor.country_code", 
                              "rank_competitor.abbreviation"]].drop_duplicates()

# Rename columns to match SQL table names
df_rankings = df_rankings.rename(columns={
    "rank_rank": "rank",
    "rank_movement": "movement",
    "rank_points": "points",
    "rank_competitions_played": "competitions_played",
    "rank_competitor.id": "competitor_id"
})

df_rankings = df_rankings[["rank", "movement", "points", "competitions_played", "competitor_id"]]

df_competitors = df_competitors.rename(columns={
    "rank_competitor.id": "competitor_id",
    "rank_competitor.name": "name",
    "rank_competitor.country": "country",
    "rank_competitor.country_code": "country_code",
    "rank_competitor.abbreviation": "abbreviation"
})



In [ ]:
df_competitors


In [ ]:
df_rankings

In [ ]:
neutral_count = (df_competitors['country'] == 'Neutral').sum()
neutral_count

In [ ]:
null_count = df_competitors['country_code'].isnull().sum()
null_count

In [ ]:
check_NEU_present = (df_competitors['country_code'] == 'NEU').sum()
check_NEU_present

We are going to replace NEU as country_code where counntrycode is not present, and we chose NEU as keyword since it represents neutral and not a duplicate word

In [ ]:
df_competitors['country_code'].fillna('NEU', inplace=True)
df_competitors

Now we run the commands to create and insert table to SQL

In [ ]:
import mysql.connector
import pandas as pd
import numpy as np

# Connect to MySQL Database
# Establish connection to MySQL 
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="240996",
    database="Tennis_project"
)

cursor = conn.cursor()

# Create Tables (Fixed "rank" Issue)
create_competitors_table = """
CREATE TABLE IF NOT EXISTS Competitors (
    competitor_id VARCHAR(50) PRIMARY KEY,
    name VARCHAR(100) NOT NULL,
    country VARCHAR(100) NOT NULL,
    country_code CHAR(3) NOT NULL,
    abbreviation VARCHAR(10) NOT NULL
);
"""

create_rankings_table = """
CREATE TABLE IF NOT EXISTS Rankings (
    rank_id INT PRIMARY KEY AUTO_INCREMENT,
    player_rank INT NOT NULL,  -- FIXED: Changed from 'rank' to 'player_rank'
    movement INT NOT NULL,
    points INT NOT NULL,
    competitions_played INT NOT NULL,
    competitor_id VARCHAR(50),
    FOREIGN KEY (competitor_id) REFERENCES Competitors(competitor_id)
);
"""
cursor.execute("DROP TABLE IF EXISTS Rankings;")
cursor.execute("DROP TABLE IF EXISTS Competitors;")

print("Tables dropped before creating")

cursor.execute(create_competitors_table)
cursor.execute(create_rankings_table)
conn.commit()

print("Tables created successfully!")

# Handle NaN values before inserting (replace NaN with None)
df_competitors = df_competitors.replace({np.nan: None})
df_rankings = df_rankings.replace({np.nan: None})

# Insert Data into Competitors Table
competitor_insert_query = """
INSERT INTO Competitors (competitor_id, name, country, country_code, abbreviation) 
VALUES (%s, %s, %s, %s, %s) 
ON DUPLICATE KEY UPDATE 
    name = VALUES(name),
    country = VALUES(country),
    country_code = VALUES(country_code),
    abbreviation = VALUES(abbreviation);
"""

competitor_data = df_competitors.to_records(index=False).tolist()  # Convert DataFrame to list of tuples
cursor.executemany(competitor_insert_query, competitor_data)
conn.commit()
print("Competitors inserted successfully!")

# Insert Data into Rankings Table
ranking_insert_query = """
INSERT INTO Rankings (player_rank, movement, points, competitions_played, competitor_id) 
VALUES (%s, %s, %s, %s, %s);
"""

ranking_data = df_rankings.to_records(index=False).tolist()  # Convert DataFrame to list of tuples
cursor.executemany(ranking_insert_query, ranking_data)
conn.commit()
print("Rankings inserted successfully!")

# Close Connection
cursor.close()
conn.close()
print("Database connection closed.")